In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import nltk   
nltk.download('punkt')
nltk.download('stopwords')
#Good reference: https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yunsun2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yunsun2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data=pd.read_csv('C:/Users/Yunsun2/Desktop/Github/yelp.csv')

In [76]:
import string
from nltk.corpus import stopwords
def preprocessing(rawtext):
    w="".join(c.lower() for c in rawtext if (c not in string.punctuation) and (not c.isdigit())) 
    
    return " ".join([i for i in w.split() if i not in stopwords.words('english')])

data['textc']=data['text'].apply(preprocessing)
print(data.head())
###takes a long time to run....

              business_id        date               review_id  stars  \
0  9yKzy9PApeiPPOUJEtnvkg  2011-01-26  fWKvX83p0-ka4JS3dc6E5A      5   
1  ZRJwVLyzEJq1VAihDhYiow  2011-07-27  IjZ33sJrzXqU-0X6U8NwyA      5   
2  6oRAC4uyJCsJl1X0WZpVSA  2012-06-14  IESLBzqUCLdSzSqm0eCSxQ      4   
3  _1QQZuf4zZOyFCvXc0o6Vg  2010-05-27  G-WvGaISbqqaMHlNnByodA      5   
4  6ozycU1RpktNG2-1BroVtw  2012-01-05  1uJFq2r5QfJG_6ExMRCaGw      5   

                                                text    type  \
0  My wife took me here on my birthday for breakf...  review   
1  I have no idea why some people give bad review...  review   
2  love the gyro plate. Rice is so good and I als...  review   
3  Rosie, Dakota, and I LOVE Chaparral Dog Park!!...  review   
4  General Manager Scott Petello is a good egg!!!...  review   

                  user_id  cool  useful  funny  \
0  rLtl8ZkDX5vH5nAx9C3q5Q     2       5      0   
1  0a2KyEL0d3Yb1V6aivbIuQ     0       0      0   
2  0hT2KtfLiobPvh6cDC8JQg     0 

In [119]:
from nltk import PorterStemmer, WordNetLemmatizer 
from nltk.corpus import wordnet  

#Function: tagging with lemmatization
lmtzr = nltk.WordNetLemmatizer().lemmatize 

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def normalize_text(text):
    word_pos = nltk.pos_tag(nltk.word_tokenize(text))
    
    lemm_words = [lmtzr(w[0], get_wordnet_pos(w[1])) for w in word_pos]  #w[0] is word, w[1] is form of the word; 
    return [x.lower() for x in lemm_words]

data['textc']=data['textc'].apply(normalize_text)
data['textc']=data['textc'].apply(lambda x: " ".join(x))

In [121]:
data.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny,textc
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0,wife take birthday breakfast excellent weather...
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0,idea people give bad review place go show plea...
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0,love gyro plate rice good also dig candy selec...
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0,rosie dakota love chaparral dog park convenien...
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0,general manager scott petello good egg go deta...


In [112]:
#For intermediate data checking; 
#Create bag of words and frequency on whole dataset
import collections
worddic=collections.Counter() 
for i in data['textc']:
    for j in i:  #i is list;  
    #for j in i.split(): #i is a paragraph; 
        worddic[j]+=1
        
print(len(worddic))  #36,271->35014 words (after removing digits), ->30521 after lemmatization; 

#print(sorted(worddic.values(), reverse=True))
#print(sum(1 for i in worddic.keys() if i.isdigit()))
#no digits now; 

30521


In [126]:
#word frequency
pd.DataFrame(sorted(worddic.values(), reverse=True)).describe(percentiles=[0.01, 0.05, 0.10, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
#remove 50% of the words with total frequency==1; 

,0
count,30521.000000
mean,21.818322
std,158.847536
min,1.000000
1%,1.000000
5%,1.000000
10%,1.000000
25%,1.000000
50%,1.000000
75%,4.000000


In [127]:
#Create bag of words and frequency on occurance of documents 
worddic2=collections.Counter() 
for i in data['textc']:
    for j in i.split(): #i is a paragraph; 
        #worddic[j]+=1


wife take birthday breakfast excellent weather perfect make sit outside overlook ground absolute pleasure waitress excellent food arrive quickly semibusy saturday morning look like place fill pretty quickly earlier get good favor get bloody mary phenomenal simply best ive ever im pretty sure use ingredient garden blend fresh order amaze everything menu look excellent white truffle scramble egg vegetable skillet tasty delicious come piece griddle bread amaze absolutely make meal complete best toast ive ever anyway cant wait go back
idea people give bad review place go show please everyone probably grip something faultthere many people like case friend arrive pm past sunday pretty crowd think sunday even think would wait forever get seat say well seat girl come back seat someone else seat waiter come get drink order everyone pleasant host seat u waiter server price good well place order decide want share baked spaghetti calzone small here beef pizza try calzone huge get small one persona

handmade instore fine candy aand chocolate awesome way well see high price owner really great people
many time pretty much cant go wrong price good service prompt friendly
use come week amaze vegetarian sandwich favorite griefo pepper jack cheese hemp cream cheese guacamole sprout lettuce black olive onion mushroom cucumber tomato hemp cream cheese die seriously also enjoy humbolt similar majic mushroom slice portabella mushroom teriyaki glaze green bell pepper pineapple pepper jack cheese inch soo big make great fill lunch cant beat price either get garlic herb bread best thing like place whole marijuana theme get old annoying location also get really really busy sandwich take time make staff usually pretty friendly social though one last thing eat wait clothes smell like food
dryrub barbecue delicious sassy barbecue sauce meh sweet good enough make
vig okay opinion meet friend happy hour friday evening pack get drinksapp special food try okay atmosphere standard trendy barmeat market

old restrant good food
love place marijuana theme isnt kidfriendly though dont bring kid bring home sub really tasty theme adult hilarious people friendly reasonably price also potato salad fantastic
use shop constantly live right across street still make drive even though price high safeway staff rock need certain type wine know find need spice thats cover every employee courteous importantly speak fluent english deli rock sock always fresh always tasty ive purchase dinner never disappoint ready cook stuff delicious mood kind meat stuff vegetable andor cheese place also love fresh food cook every day right store bbq massive sunday omelet choices plenty cheap certainly get money worth go early want breakfast run quick buy good beer bottle dogfish head rock
since move arizona select frys new grocer happy choice immediately sign vip card love see many saving list receipt every trip especially happy move closer frys open hour even shop wee hour morning expect see nocturnal asu student int

exactly hole wall pho place like totally remind la sf authentic pho place multicultural city congrats phoenix completely white breed redneck must give thanks center city anyhow portion large ingredient taste fresh price inexpensive youre one shelter folk get freak rude bum lack decor stay shelter hoa tract home go mall
never write negative review unfortunately think place need get together location never seem successful drug husband excite try hop see improvement luck vegetarian entree really kid friendsly small portion pricey look even love support new restaurant wont back hope figure make work
ive mojo fan since well since mojo first become mojo yogurtology open right street house arch skeptical brow decide give mojowannabe try long sigh newfound love yogurtology feel little guilty ive replace old love new one life suppose fact yogurtology wide range excellent flavor freeze yogurt seem rotate daily ive see chocolate mint oreo cupcake batter red velvet chocolate peanut butter mango bl

service slow horrible get steak egg steak half fat waiter response oh new supplier didnt take back charge u way egg cold stay away place eat bar look happen
one favorite bar area bar good vibe food good bartender attentive price reasonable good time without dig deeply pocket
deli style restaurant everything want good service invite atmosphere super clean appearance freshly prepare food attractive delicious salmon blt even ask friendly order taker make bacon crisp salmon slightly char everything wonderful sandwhich perfect honest would switch shredded iceberg lettuce spinach leave still sandwhich terrific nice proportion aioli sauce bring together din comanion spinach sirloin salad also completely finished end meal look empty bowl come must really good item menu look forward try next time like wood fire pizzas burger great location heart old town gilbert gallery small business area great find definitely go back
really want like honey bear far bbq go perfectly average meal side salad ok 

efficient friendly service good freshsqueezed juice rarely eat breakfast food enjoy new mexico skillet
eh write review matt go back thankfully wont hard recall experience place small food fantastic wait usually bathroom adventure didnt long wait go entirely luck ive see crazy wait par course food good local good service fast spot bathroom next door old motel apparently separate room set aside matts one stand feature standout sort place
love joes farm grillyou havent live tried blue cheese burgernot sure name menui fancy steak encrust blue cheese burger next best thing garlic fry die forheck everything cook incredibleworth drive
best pizza valley ive ive yet pizzaria banco great service always make fast great pricingdoenst travel well youre gon na car wouldnt recommend iti think thats there lot soggy crust comment etc
food consistently good tend come annually christmastime shopping mall go last week lady ultimate food envy watch cohort eat chicken salad sandwich come back week test best

love place cute trendy great food love crepe fav bbq pork sandwich flavorful avocado pastry great great place lunch afternoon tea
work area come maybe month pho crave need place near work decor relax music pleasant find service fast really important lunch one side price little high side pho somewhat bland side lot american coworkers wont go mom pop pho house pho dalat somewhere would okay youre side town pho crave pho dalat definitely cure dont get expect wow complex flavor broth
go last night search restaurant good soup instead decide try texas roadhouse soon walk feel welcome right home staff smile welcome atmosphere place great roast peanut await table along delicious bread bacon cheese burger could finish half husband steak actually enjoy waitress kept drink check u enough much bothering service food great definitely recommend others back
loyal durant feel like heelbut city hall divine steak side drinksthe thing didnt dessertbut later could afford
great fresh food vegetarian option

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.



know could still creative since year since do anything involve even minor level creativity fun girl friend take initiative organize kitchen tea recent saturday afternoon table girl friend schedule pottery make initial idea venue admittedly little different good expect many way one get dirty dirty really also mind dirty wonder whether go go two able bring along food drink make excellent responsibility share win din arena set party arrangement snack champagne inspired start artistic journey idea friend make dinner plate item bride bes choice give wedding present think idea really original bride select dinner plate choice brief friendly sweet staff member different effect paint plate bake informed go select different color need different art work everybody start paint one u get really involve creation event take u much longer anticipated good way finish hour start much enlighten activity everybody agree go sheer enjoyment good feeling togetherness promote yet see end product one friends 

go cricket pavilion see jack johnson great show fair venue way cool show staff come around hand green recycle bag can bottle jack arty take note drink park lot show tailgater around u pretty nice crowd overall hassle way show june august thank god weather break cool get many people together dance around generate heat typical venue price high mixed drink beersthey pabst blue ribbon tall boys ha score case pbr nutshell great show good weather good crowdfree park awesome see rockn metal fest middle summer wouldnt
place great grow alot food use grow buy local food taste greatdont forget get milkshake also great place sit eat outside nice
mixed review husband comment salad good spinach gorgonzola salad pizza really good expensive plus thin type wouldnt expect pay much wait staff good ambiance pretty nice ny type feeling pricey go back comment salad much dressing drip balsamic stuff dont like soppy lettuce pizza ok serve u fairly quickly consider place pack however hot plus wave server ask h

In [ ]:
'''
train_bow=CountVectorizer().fit(train['text'])
train_bow.get_feature_names()
'''
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import string
from nltk.corpus import stopwords


pipe=Pipeline([('bow', CountVectorizer()),
               ('tfidf', TfidfTransformer()), 
               ('model', MultinomialNB())])  

X=data['textc']
Y=data['stars']
rs=9876
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.3, random_state=rs) 

pipe.fit(xtrain, ytrain)
pred=pipe.predict(xtest)
print('predictions with default hyparameters for Test'+'\n')
print('Accuracy')
print(accuracy_score(ytest, pred, normalize=True))
print(confusion_matrix(ytest, pred))
print(classification_report(ytest, pred))  